In [1]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import os
from io import BytesIO
from datetime import datetime
from st_social_media_links import SocialMediaIcons
import base64
from streamlit_option_menu import option_menu

2025-03-10 23:24:22.233 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
endeksler=pd.read_csv("endeksler.csv",index_col=0)
endeksler.index=pd.to_datetime(endeksler.index)
endeksler_aylık=pd.DataFrame(columns=endeksler.columns)


In [20]:
endeksler.to_csv("endeksler_int.csv")

In [13]:
ağırlıklar=pd.read_csv("ağırlıklartüfe.csv",index_col=0)
ağırlıklar.loc["Benzin"]

Ağırlık                0.012062
Ana Grup              Ulaştırma
Endeks_2024-12-31         100.0
Endeks_2025-01-06    102.097821
Endeks_2025-01-07    105.015647
                        ...    
Endeks_2025-03-06    105.045436
Endeks_2025-03-07    100.467038
Endeks_2025-03-08    100.467038
Endeks_2025-03-09    100.467038
Endeks_2025-03-10    100.498892
Name: Benzin, Length: 66, dtype: object

In [19]:
endeksler["Tramvay Ücreti"].loc[:"2025-01-06"]=100
endeksler.to_csv("endeksler.csv")

In [8]:
endeksler_aylık=endeksler_aylık.T

In [11]:
endeksler_aylık.columns=endeksler_aylık.iloc[0,:]
endeksler_aylık=endeksler_aylık.iloc[1:]

In [15]:
endeksler_aylık=endeksler_aylık.reset_index()
endeksler_aylık=endeksler_aylık.rename(columns={"index":"Madde"})

In [16]:
endeksler_aylık

Tarih,Madde,2025-02,2025-03
0,Ampul,4.786673,8.12935
1,Ankastre Fırın,1.613256,4.148246
2,Ankastre Mikrodalga Fırın,1.889752,5.738414
3,Ankastre Ocak (Set Üstü),2.523521,4.461977
4,Antep Fıstığı,-0.151343,0.308796
...,...,...,...
320,İç çamaşırı (Çocuk için),4.81325,6.074393
321,Şeftali,2.478854,4.822595
322,Şehriye,2.599573,3.16732
323,Şemsiye,34.135809,44.814717


In [3]:
from datetime import datetime,timedelta
import pytz
tüfe=pd.read_csv("tüfe.csv",index_col=0)
tüfe.index=pd.to_datetime(tüfe.index)
gfe1=tüfe.copy()
gfe1["Date"]=pd.to_datetime(gfe1.index)
gfe1["Ay"]=gfe1["Date"].dt.month
gfe1["Yıl"]=gfe1["Date"].dt.year    
month = gfe1["Ay"].iloc[-1]
year=gfe1["Yıl"].iloc[-1] 
oncekiyear=gfe1["Yıl"].iloc[-1] 

In [4]:
def hareketli_aylik_ortalama(df):
        değer = df.name  # Kolon ismi
        df = pd.DataFrame(df)
        df["Tarih"] = pd.to_datetime(df.index)  # Tarih sütununu datetime formatına çevir
        df["Gün Sırası"] = df.groupby(df["Tarih"].dt.to_period("M")).cumcount() + 1  # Her ay için gün sırasını oluştur
        
        # Her ay için ilk 24 günü sınırla ve hareketli ortalama hesapla
        df["Aylık Ortalama"] = (
            df[df["Gün Sırası"] <= 24]
            .groupby(df["Tarih"].dt.to_period("M"))[değer]
            .expanding()
            .mean()
            .reset_index(level=0, drop=True)
        )
        
        # Orijinal indeksi geri yükle
        df.index = pd.to_datetime(df.index)
        return df
    

   

tarihim=pd.to_datetime(gfe1.index[-1]).day
if tarihim>24:
    tarihim=24
if tarihim<10:
    tarihim="0"+str(tarihim)


def hareketli_aylik_ortalama1(df):
        değer=df.name
        df=pd.DataFrame(df)
        df["Tarih"]=pd.to_datetime(df.index)
        df['Aylık Ortalama'] = df.groupby(df['Tarih'].dt.to_period('M'))[değer].expanding().mean().reset_index(level=0, drop=True)
        df.index=pd.to_datetime(df.index)
        return df



hareketlima = hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill")
from datetime import datetime,timedelta
tarih=datetime.now().strftime("%Y-%m")

onceki=(datetime.now()-timedelta(days=31)).strftime("%Y-%m")
hareketliartıs=hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[tarih:]/hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[:f"{onceki}-24"]
hareketliartıs=(hareketliartıs-1)*100

C:\Users\Bora\AppData\Local\Temp\ipykernel_31400\2297976651.py:40: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hareketlima = hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill")
C:\Users\Bora\AppData\Local\Temp\ipykernel_31400\2297976651.py:45: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hareketliartıs=hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[tarih:]/hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[:f"{onceki}-24"]
C:\Users\Bora\AppData\Local\Temp\ipykernel_31400\2297976651.py:45: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hareketliartıs=hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(metho

In [7]:
from datetime import datetime,timedelta
tarih=datetime.now().strftime("%Y-%m")
onceki=(datetime.now()-timedelta(days=31)).strftime("%Y-%m")

In [12]:
cari=hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[tarih:]
hareketliartıs=cari.values/hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[f"{onceki}-1":f"{onceki}-24"].iloc[:len(cari)].values
hareketliartıs=pd.Series(hareketliartıs,index=cari.index)
hareketliartıs=(hareketliartıs-1)*100

C:\Users\Bora\AppData\Local\Temp\ipykernel_5540\62457178.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cari=hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[tarih:]
C:\Users\Bora\AppData\Local\Temp\ipykernel_5540\62457178.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hareketliartıs=cari.values/hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[f"{onceki}-1":f"{onceki}-24"].iloc[:len(cari)].values


In [14]:
cari=hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill")
aylıkdata=cari.resample('M').last().pct_change().loc["2025-02":]*100
aylıkdata.iloc[-1]=hareketliartıs.iloc[-1]
aylıkdata

C:\Users\Bora\AppData\Local\Temp\ipykernel_5540\2097899177.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cari=hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill")
C:\Users\Bora\AppData\Local\Temp\ipykernel_5540\2097899177.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  aylıkdata=cari.resample('M').last().pct_change().loc["2025-02":]*100


2025-02-28    3.45358
Freq: ME, Name: Aylık Ortalama, dtype: float64

In [25]:
["Tarih"].extend(endeksler.columns)

In [32]:
gruplar=pd.read_csv("gruplar_int.csv",index_col=0)
gruplar.index=pd.to_datetime(gruplar.index)
gruplar=gruplar.sort_index()
gruplar_aylık=pd.DataFrame(columns=gruplar.columns)
for col in gruplar.columns:
    cari=hareketli_aylik_ortalama(gruplar[col])["Aylık Ortalama"].fillna(method="ffill")
    gruplar_aylık[col]=cari.resample('M').last().pct_change().loc["2025-02":]*100
    carim=hareketli_aylik_ortalama(gruplar[col])["Aylık Ortalama"].fillna(method="ffill").loc[tarih:]
    hareketliartıs=carim.values/hareketli_aylik_ortalama(gruplar[col])["Aylık Ortalama"].fillna(method="ffill").loc[f"{onceki}-1":f"{onceki}-24"].iloc[:len(carim)].values
    hareketliartıs=pd.Series(hareketliartıs,index=carim.index)
    hareketliartıs=(hareketliartıs-1)*100
    gruplar_aylık[col].iloc[-1]=hareketliartıs.iloc[-1]
    gruplar_aylık=pd.DataFrame(gruplar_aylık)
gruplar_aylık["Tarih"]=(gruplar_aylık.index.strftime("%Y-%m"))
cols=["Tarih"]
cols.extend(gruplar.columns)
gruplar_aylık=gruplar_aylık[cols]
gruplar_aylık=gruplar_aylık.reset_index(drop=True)

C:\Users\Bora\AppData\Local\Temp\ipykernel_5540\3622076335.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cari=hareketli_aylik_ortalama(gruplar[col])["Aylık Ortalama"].fillna(method="ffill")
C:\Users\Bora\AppData\Local\Temp\ipykernel_5540\3622076335.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  gruplar_aylık[col]=cari.resample('M').last().pct_change().loc["2025-02":]*100
C:\Users\Bora\AppData\Local\Temp\ipykernel_5540\3622076335.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  carim=hareketli_aylik_ortalama(gruplar[col])["Aylık Ortalama"].fillna(method="ffill").loc[tarih:]
C:\Users\Bora\AppData\Local\Temp\ipykernel_5540\3622076335.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill()

In [33]:
gruplar_aylık

,Tarih,Ev eşyası,Eğitim,Eğlence ve kültür,Giyim ve ayakkabı,Gıda ve alkolsüz içecekler,Haberleşme,Konut,Lokanta ve oteller,Ulaştırma,Çeşitli mal ve hizmetler,TÜFE
0,2025-02,5.134547,7.385641,3.67301,4.427764,3.36911,-1.146261,3.004544,3.917252,2.107484,5.222684,3.45358


In [32]:
cari=hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[tarih:]

C:\Users\Bora\AppData\Local\Temp\ipykernel_46100\1288260787.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cari=hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[tarih:]


In [33]:
hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[f"{onceki}-1":f"{onceki}-24"].iloc[:len(cari)]

C:\Users\Bora\AppData\Local\Temp\ipykernel_46100\2356160772.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[f"{onceki}-1":f"{onceki}-24"].iloc[:len(cari)]


2025-01-01    100.532588
2025-01-02    100.609943
2025-01-03    100.690320
2025-01-04    100.753498
2025-01-05    100.806831
2025-01-06    100.864572
2025-01-07    100.930172
2025-01-08    100.997635
2025-01-09    101.067898
2025-01-10    101.130934
Name: Aylık Ortalama, dtype: float64

In [35]:
cari.values/hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[f"{onceki}-1":f"{onceki}-24"].iloc[:len(cari)].values

C:\Users\Bora\AppData\Local\Temp\ipykernel_46100\1253615930.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cari.values/hareketli_aylik_ortalama(tüfe.iloc[:,0])["Aylık Ortalama"].fillna(method="ffill").loc[f"{onceki}-1":f"{onceki}-24"].iloc[:len(cari)].values


array([1.02997922, 1.02961507, 1.02925488, 1.02953024, 1.03005078,
       1.03059827, 1.03087737, 1.03109726, 1.03112856, 1.03126141])